# D3 integration in Jupyter notebook

### Goal: 

The main goal is to integrate D3 in jupyter notebook and achieve two way communication between front end javascript and backend python notebook. This will allow us to get the data from user interaction.


### Libraries:

The main component we will be using is ipywidgets. You can learn more about this at http://ipywidgets.readthedocs.io/en/latest/.
Using widgets we can build interactive visualizations in notebooks.
 
Apart from ipywidgets, othe libraries we will be using are pandas for data manupulation, Ipython.display to display the widget.

In [17]:
import ipywidgets as widgets
from IPython.display import display
from traitlets import Unicode, validate, List
import pandas as pd

We will be following the example on building a custom widget from http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Custom.html

#### Import d3 using require

In [18]:
%%javascript
require.config({
    paths: {
        d3: 'https://d3js.org/d3.v4.min'
    }
});

<IPython.core.display.Javascript object>

## Example 1: Bar Chart - Selection
We will be using the soccer player data set for all of our examples.

The data contains top 50 players and attributes like overall_score, crossing, finishing etc. over the years 2007-16. We will mostly use the overall_score attribute.

In the first chart, we will show top 5 players with maximum overall score averaged over the years as a bar chart. If user clicks on any one bar chart, then we will show the progress of the player over the years.

In [3]:
players_data = pd.read_csv(open("player_data.csv"))
data_average = players_data.groupby('player_name')['overall_rating'].mean().reset_index(name='average_score')
data_average_top5 = data_average.head(5)

First, create a back-end python class for our widget.

In [4]:
class BarWidget(widgets.DOMWidget):
    _view_name = Unicode('BarView').tag(sync=True)
    _view_module = Unicode('barChart').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    value = List([]).tag(sync=True)
    player_name = Unicode('').tag(sync=True)

Next, we will create the front-end backbone view. The view name should be same as that of back-end class. This helps the
widget framework to link both.

The render method is where our javascript code lies. We will use this section to render our d3 chart.

In [5]:
%%javascript
require.undef('barChart');

define('barChart', ["@jupyter-widgets/base", "d3"], function(widgets, d3) {

    var BarView = widgets.DOMWidgetView.extend({

        render: function() {
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
        },
         value_changed: function() {
            console.log("Inside bar value_changed");
            var data = this.model.get('value');
            var self = this;
            console.log("data is ",data)
            var margin = {top: 20, right: 20, bottom: 30, left: 40},
            width = 400 - margin.left - margin.right,
            height = 300 - margin.top - margin.bottom;

            // set the ranges
            var x = d3.scaleBand()
                      .range([0, width])
                      .padding(0.1);
            var y = d3.scaleLinear()
                      .range([height, 0]);
            $("#barChart").remove();
            this.$el.append("<div id='barChart'></div>");
            $("#barChart").width("400px");
            $("#barChart").height("300px");
            // console.log(d3);
            var svg = d3.select("#barChart").append("svg")
                        .attr("width", width + margin.left + margin.right)
                        .attr("height", height + margin.top + margin.bottom)
                        .append("g")
                        .attr("transform", 
                              "translate(" + margin.left + "," + margin.top + ")");
            
            x.domain(data.map(function(d) { return d.player_name; }));
            y.domain([0, d3.max(data, function(d) { return d.average_score; })]);
            console.log(svg);
            // append the rectangles for the bar chart
            svg.selectAll(".bar")
                .data(data)
                .enter().append("rect")
                .attr("class", "bar")
                .attr("x", function(d) { return x(d.player_name); })
                .attr("width", x.bandwidth())
                .attr("y", function(d) { return y(d.average_score); })
                .attr("height", function(d) { return height - y(d.average_score); })
                .style("fill","steelblue")
                .on("click", function(d){
                    d3.selectAll('.bar').style("fill","steelblue");
                    d3.select(this).style("fill","green");
                    self.model.set('player_name', d.player_name);
                    self.model.save_changes();
                    self.touch();
                });

            // add the x Axis
            svg.append("g")
            .attr("transform", "translate(0," + height + ")")
            .call(d3.axisBottom(x));

            // add the y Axis
            svg.append("g")
            .call(d3.axisLeft(y));
            
        },
    });

    return {
        BarView : BarView
    };
});

<IPython.core.display.Javascript object>

Lets create the widget for Line chart to show the progress of a player whose bar is selected.

For this again, we need to create the back-end python model.

In [6]:
class LineWidget(widgets.DOMWidget):
    _view_name = Unicode('LineView').tag(sync=True)
    _view_module = Unicode('lineChart').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    value = List([]).tag(sync=True)
    selected_years = List([]).tag(sync=True)

Now, the front-end view for the line chart

In [8]:
%%javascript
require.undef('lineChart');
define('lineChart', ["@jupyter-widgets/base", "d3"], function(widgets, d3) {

    var LineView = widgets.DOMWidgetView.extend({

        render: function() {
            this.value_changed();
//             this.model.on('change:value', this.value_changed, this);
            this.listenTo(this.model, 'change:value', this.value_changed, this);
        },

        value_changed: function() {
            var player = this.model.get('value');
            var that = this;
            var test = this.model.get('count');
//             alert(test);
var yearValues = [];
var attribValues = [];
var playerYearDataList = [];
player.sort(function(x, y){
    return d3.ascending(x[2], y[2]);
})
console.log(player);
player.forEach(function(d){
    yearValues.push(d["year"]);
    attribValues.push(d["overall_rating"]);
})
console.log(yearValues);
console.log(attribValues);
var margin = {top: 10, right: 30, bottom: 30, left: 50};
var svgHeight = 400;
var svgWidth = 1000;
  //create canvas
  $("#chart1").remove();
  this.$el.append("<div id='chart1'></div>");
  $("#chart1").width("960px");
  $("#chart1").height("400px");        
  var margin = {top: 20, right: 20, bottom: 30, left: 40};
  var width = 880 - margin.left - margin.right;
  var height = 500 - margin.top - margin.bottom;
  var svg = d3.select("#chart1").append("svg")
    .style("position", "relative")
    .style("max-width", "960px")
    .attr("width", width + "px")
    .attr("height", (height + 50) + "px");
  svg.append('g')
     .attr("id", "xAxis");
  svg.append('g')
     .attr("id", "yAxis");
    console.log(d3);
  let yScale = d3.scaleLinear()
            .domain([d3.min(attribValues, d => d), d3.max(attribValues, d => d)])
            .range([svgHeight - margin.top - margin.bottom, 0]);

        let yAxis = d3.axisLeft();
        // assign the scale to the axis
        yAxis.scale(yScale);


        var yAxisG = d3.select("#yAxis")
            .attr("transform", "translate("+margin.left+"," + margin.top +")");
        // self.svg.append("g")
        // .attr("class" , "yAxis");

        console.log(yAxisG);

        yAxisG.transition(3000).call(yAxis);


        let xScale = d3.scaleLinear()
            .domain([d3.min(yearValues), d3.max(yearValues)])
            .range([0, 600]);

        let xAxis = d3.axisBottom();
        // assign the scale to the axis
        xAxis.scale(xScale);


        var xAxisG = d3.select("#xAxis")
            .attr("transform", "translate("+(margin.left+ 10)+"," + (svgHeight - margin.bottom) +")");
        // self.svg.append("g")
        // .attr("class" , "yAxis");

        console.log(xAxisG);
//         let color = d3.scaleLinear()
//             .domain([0, playerYearDataList.length])
//             // .range(["#016450", "#ece2f0"]);
//             .range(["#2019F6", "#F61936"]);

        xAxisG.transition(3000).call(xAxis);
        svg.selectAll(".playerPath").remove();
        svg.selectAll(".playerNode").remove();
            console.log(player.name);
            console.log(player.playerYearData);

            var lineCoords = []
            for(var k=0; k<yearValues.length; k++){
                lineCoords.push([xScale(yearValues[k]), yScale(attribValues[k])]);
            }

            console.log(lineCoords);

            var lineGenerator = d3.line();
            var pathString = lineGenerator(lineCoords);

            console.log(pathString);

            svg.append('path')
                .attr('d', pathString)
                .attr("transform", "translate("+(margin.left+ 10)+"," + (margin.top) +")")
                .attr("style", "fill : none;")
                .attr("class", "playerPath")
                .style("stroke", "steelblue")
                .style("stroke-width", 3)
                .style('opacity', 0.5);
            
            lineCoords.forEach(function(point){
                svg.append('circle').attr('cx', point[0])
                    .attr("cy", point[1])
                    .attr("r", 5)
                    .attr("transform", "translate("+(margin.left+ 10)+"," + (margin.top) +")")
                    .attr("class", "playerNode");
            });
            d3.selectAll(".brush").remove();
        var brush = d3.brushX().extent([[margin.left,svgHeight-margin.bottom-20],[svgWidth,svgHeight-10]]).on("end", brushed);

        svg.append("g").attr("class", "brush").call(brush);


        function brushed() {

            var sel = d3.event.selection;

            if(sel === null){
                return;
            }

            var yearValuesBrushed = yearValues.filter((d) => xScale(d)+margin.left+ 10 >= sel["0"] &&  xScale(d)+margin.left+ 10  <= sel["1"]);

            

            console.log(yearValuesBrushed);
            that.model.set('selected_years', yearValuesBrushed);
            that.model.save_changes();
            that.touch();
//             alert(that.model.get('test'));

            // var dataSel = self.posd.filter((d) => d.position >= sel["0"] && d.position <= sel["1"]);
            // window.selectedStatesIn = dataSel.map( d => d.elem);
            // self.shiftChart.update(window.selectedStatesIn, window.selectedYearsIn);
//             self.updateBars(playerYearDataList, uniqueYrs , attrib, color);


        }
        },
    });

    return {
        LineView : LineView
    };
});

<IPython.core.display.Javascript object>

In [38]:
barWidget = BarWidget()
def updateBar():
    barWidget.value = data_average_top5.to_dict(orient='records')
display(barWidget)
updateBar()

BarWidget()

In [39]:
barWidget.player_name

'Arjen Robben'

We need to initialize the line widget before using it. Also updateLineChart function filters the data based on the name.

In [40]:
lineWidget = LineWidget()
def updateLineChart(name):
    filterByName = players_data[players_data["player_name"]==name]
    jsonValue = filterByName[["player_name", "overall_rating", "year"]]
    lineWidget.value = jsonValue.to_dict(orient='records')

After selecting any player from above bar chart, we will run the following cell to show the progress of him over the years.

In [43]:
display(lineWidget)
updateLineChart(barWidget.player_name)

LineWidget(value=[{'player_name': 'Andres Iniesta', 'overall_rating': 83.5, 'year': 2007}, {'player_name': 'Andres Iniesta', 'overall_rating': 86.0, 'year': 2008}, {'player_name': 'Andres Iniesta', 'overall_rating': 86.0, 'year': 2009}, {'player_name': 'Andres Iniesta', 'overall_rating': 87.0, 'year': 2010}, {'player_name': 'Andres Iniesta', 'overall_rating': 89.0, 'year': 2011}, {'player_name': 'Andres Iniesta', 'overall_rating': 90.5, 'year': 2012}, {'player_name': 'Andres Iniesta', 'overall_rating': 89.75, 'year': 2013}, {'player_name': 'Andres Iniesta', 'overall_rating': 89.0, 'year': 2014}, {'player_name': 'Andres Iniesta', 'overall_rating': 88.0, 'year': 2015}])

## Example 2: Bar chart - Filter

This example shows the implementation of filtering.

The initial bar chart shows the average score of all 50 players. We will have a text box to enter a number, then the plot will be updated to show the players having average score of more than or equal to the entered number. 

In [44]:
barChart2 = BarWidget()
def updateBarChart2(score):
    player_with_score = data_average[data_average['average_score']>score['new']]
    barChart2.value = player_with_score.to_dict(orient='records')
def createScoreTextBox():
    score_text_box = widgets.widgets.FloatText(
    value=0,
    description='Show player with score more than or equal to:',
    disabled=False
    )
    score_text_box.observe(updateBarChart2, names='value')
    display(score_text_box)
createScoreTextBox()
display(barChart2)
updateBarChart2({'new':0})

FloatText(value=0.0, description='Show player with score more than or equal to:')

BarWidget()

### Example 3 - Range selection using brush

The goal is to get the brush selection in d3 plot to python side. It is achieved in the same way as above. We will create another varuable (trait) and update it in front-end.

The following line chart shows the progress of player Messi. Then we will print the selected years to make sure that we recieve the range

In [45]:
lineWidget2 = LineWidget()
def showProgressOfPlayer(name):
    filterByName = players_data[players_data["player_name"]==name]
    returnData = filterByName[["player_name", "overall_rating", "year"]]
    lineWidget2.value = returnData.to_dict(orient='records')
display(lineWidget2)
showProgressOfPlayer("Lionel Messi")

LineWidget()

In [46]:
lineWidget2.selected_years

[2010, 2011, 2012]